In [1]:
# pip install awswrangler

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import awswrangler as wr

sys.path.append(os.path.dirname(os.getcwd())+'/src')
from preprocess import preprocess_module

In [3]:
preproc = preprocess_module()
preproc.gen_time, preproc.class_name

(datetime.datetime(2022, 8, 19, 13, 32, 57, 684765), 'preprocessing module')

---
## Reading input data

In [4]:
df_fragments = preproc.get_fragments()
df_fragments.head()

,MpoCentro,Texto Limpo
0,cosr-nco,Estabelecer procedimentos para o controle da t...
1,cosr-nco,2.1. Os procedimentos para a reali...
2,cosr-nco,1. OBJETIVO2....................................
3,cosr-nco,1. OBJETIVO2....................................
4,cosr-nco,1. OBJETIVO2....................................


In [9]:
def regex_subs_fragments(
    text
    ):
    
    import regex as re

    # seções numéricas
    rule_list = []
    subsection_max_num = 4
    for i in range(subsection_max_num+1, 2,-1):
        rule_list.append('[0-9]+.'*i)
        rule = '[0-9]+.'*i
        rule_list.append(rule[0:-1])

    for rule in rule_list:
        text = re.sub(f'{rule}', '', text)

    # seções alfabéticas
    text = re.sub(r'(\s[abcde]\)\s+)','. ',text)
    text = re.sub(r'([0-9]\)\s+)|([0-9][0-9]\)\s+)','. ',text)

    # módulos alfanuméricos
    text = re.sub(r'(módulo\s[0-9])|(modulo\s[0-9])|(submódulo\s[0-9].[0-9][0-9].)|(submodulo\s[0-9].[0-9][0-9].)','',text)
    text = re.sub(r'(módulo\s[0-9])|(modulo\s[0-9])|(submódulo\s[0-9])|(submodulo\s[0-9])','',text)
    text = re.sub(r'(módulo)|(submódulo)|(modulo)|(submodulo)','',text)

    # caracteres especiais
    text = re.sub(r'(•)', "", text)
    text = re.sub(r'(\s-\s)', " ", text)
    text = re.sub(r'(-\s)', " ", text)

    # palavras específicas
    text = re.sub(r'((objetivo)[1-9].|(considerações gerais)[1-9].)',(''),text)

    # separação de termos - concatenação indevida
    text = re.sub(' +', ' ', text)
    text = re.sub(r'([a-zzáàâãéèêíïóôõöúçñ][a-zzáàâãéèêíïóôõöúçñ])([0-9])', r'\1 \2', text)
    text = re.sub(r'([0-9])(energização)', r'\1 \2', text)
    text = re.sub(r'([0-9])(desenergização)', r'\1 \2', text)
    text = re.sub(r'([a-rzáàâãéèêíïóôõöúçñ])(energização)', r'\1 \2', text)
    text = re.sub(r'([t-zzáàâãéèêíïóôõöúçñ])(energização)', r'\1 \2', text)
    text = re.sub(r'([a-rzáàâãéèêíïóôõöúçñ])(desenergização)', r'\1 \2', text)
    text = re.sub(r'([t-zzáàâãéèêíïóôõöúçñ])(desenergização)', r'\1 \2', text)
    text = re.sub(r'([a-zzáàâãéèêíïóôõöúçñ])(manobra)', r'\1 \2', text)
    text = re.sub(r'([a-zzáàâãéèêíïóôõöúçñ])(lt)', r'\1 \2', text)
    text = re.sub(r'[^\S]?(\(.*?\))[^\S]?', r' \1 ', text)
    text = re.sub(r"([\w/'+$\s-]+|[^\w/'+$\s-]+)\s*", r"\1 ", text)
    text = re.sub('(&#xa0;)', ' ', text)
    text = re.sub('(\\xa0)', ' ', text)
    text = re.sub('(color:#ffffff)', ' ', text)
    text = re.sub('<\span>', ' ', text)
    text = re.sub('\uf0b7', ' ', text)
    text = re.sub('\uf044', ' ', text)

    # substituir padrões de colunas temporárias para separadores
    text = re.sub(r'-[0-9][a-z][a-z][a-z][a-z]-[a-z][a-z][a-z]','#',text)
    text = re.sub(r'-[0-9][a-z][a-z][a-z][a-z]-[a-z][a-z]','#',text)
    text = re.sub(r'[a-z][a-z][a-z][a-z]-[a-z][a-z][a-z]','#',text)
    text = re.sub(r'[a-z][a-z][a-z][a-z]-[a-z][a-z]','#',text)
    text = re.sub(r'[a-z][a-z][a-z]-[a-z][a-z]','#',text)
    text = re.sub(r'chesf (cron)','#',text)
    text = re.sub(r'execuçãoprocedimentoitem / [a-z][a-z][a-z]-[a-z][a-z]','#',text)
    text = re.sub(r'execuçãoprocedimentoitem / [a-z][a-z][a-z]-[a-z][a-z][a-z]','#',text)
    text = re.sub(r'passocoordenaçãocontrolecomando','#',text)
    text = re.sub(r'execuçãoprocedimentosobjetivo','#',text)
    text = re.sub(r'execuçãoprocedimento','#',text)
    text = re.sub('#', '', text)

    # remover excesso de espaços
    text = re.sub(' +', ' ', text)

    # remoção de pontos repetidos
    text = text.split('.')
    text = [i for i in text if i!='']
    text = '.'.join(text)

    return text


def preprocess_fragments(
    df, 
    col
    ):

    df['processed'] = df[col].apply(lambda x:regex_subs_fragments(x))

    return df

In [10]:
df_fragments = preprocess_fragments(df_fragments, 'Texto Limpo')
df_fragments.head()

,MpoCentro,Texto Limpo,processed
0,cosr-nco,Estabelecer procedimentos para o controle da t...,Estabelecer procedimentos para o controle da t...
1,cosr-nco,2.1. Os procedimentos para a reali...,2 . 1 . Os procedimentos para a realização de ...
2,cosr-nco,1. OBJETIVO2....................................,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...
3,cosr-nco,1. OBJETIVO2....................................,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...
4,cosr-nco,1. OBJETIVO2....................................,1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPA...


In [14]:
test = df_fragments['processed'].iloc[3]

In [25]:
test

' 1 . OBJETIVO2 . CONSIDERAÇÕES GERAIS3 . PREPARAÇÃO PARA MANOBRAS EM LINHAS DE TRANSMISSÃO3 . 1 . LT 230 kV UTE Jaguatirica II / Boa Vista Desenergização da LT 230 kV UTE Jaguatirica II / Boa Vista ( ambos os sentidos ) Energização da LT 230 kV UTE Jaguatirica II / Boa Vista 3 . 2 . LT 69 kV Boa Vista / Monte Cristo Desenergização da LT 69 kV Boa Vista / Monte Cristo Energização da LT 69 kV Boa Vista / Monte Cristo 3 . 3 . LT 69 kV Boa Vista / Distrito com derivação para a SE Equatorial e para a UTE Monte Cristo Bloco II Desenergização da LT 69 kV Boa Vista / Distrito com derivação para a SE Equatorial e para a UTE Monte Cristo Bloco II Energização da LT 69 kV Boa Vista / Distrito com derivação para a SE Equatorial e para a UTE Monte Cristo Bloco 23 . 4 . LT 69 kV Boa Vista / Floresta C1 com derivação para SE Satélite Desenergização da LT 69 kV Boa Vista / Floresta C1 com derivação para SE Satélite Energização da LT 69 kV Boa Vista / Floresta C1 com derivação para SE Satélite 3 . 5 . 

In [16]:
import regex as re
rule = re.compile('lt', flags=re.IGNORECASE)

In [34]:
result = re.search('potência', 'potência', re.IGNORECASE)
result[0]

'potência'

In [27]:
result

<regex.Match object; span=(98, 100), match='LT'>

In [5]:
# df_equipments = preproc.get_equipments()
# df_equipments.head()

In [6]:
# df_equipments_fameq = preproc.get_equipments_fameq()
# df_equipments_fameq.head()

There is no interesction between guid_tipoequipamentosgi and guid_equipamento:

In [7]:
# pd.merge(left=df_equipments, right=df_equipments_fameq, left_on='guid_equipamento', right_on='guid_tipoequipamentosgi', how='inner')

---
## Preprocessing datasets

In [8]:
df_fragments = preproc.preprocess_fragments(df_fragments, 'Texto Limpo')
df_fragments.head()

,MpoCentro,Texto Limpo,processed
0,cosr-nco,estabelecer procedimentos para o controle da t...,estabelecer procedimentos para o controle da t...
1,cosr-nco,2.1. os procedimentos para a reali...,2 . 1 . os procedimentos para a realizacao de ...
2,cosr-nco,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...
3,cosr-nco,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...
4,cosr-nco,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...


In [9]:
# df_equipments = preproc.preprocess_equipments(df_equipments, 'nom_longoequipamento')
# df_equipments.head()

---

## Marking input data

### Marking equipments

In [10]:
df_fragments_eq = preproc.find_equipments(df=df_fragments, col='processed')
df_fragments_eq.astype('str').drop_duplicates(subset=['MpoCentro', 'processed', 'EQUIPAMENTO'], keep='last', inplace=True)
df_fragments_eq.head()

,MpoCentro,Texto Limpo,processed,EQUIPAMENTO
0,cosr-nco,estabelecer procedimentos para o controle da t...,estabelecer procedimentos para o controle da t...,[]
1,cosr-nco,2.1. os procedimentos para a reali...,2 . 1 . os procedimentos para a realizacao de ...,[]
2,cosr-nco,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...,"[lt 69 kv boa vista / monte cristo, lt 230 kv ..."
3,cosr-nco,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...,"[lt 69 kv boa vista / monte cristo, lt 230 kv ..."
4,cosr-nco,1. objetivo2....................................,1 . . preparacao para manobras em linhas de t...,"[lt 69 kv boa vista / monte cristo, lt 230 kv ..."


In [10]:
samples = preproc.create_synthetic_samples(
    df_fragments_eq,
    entity_col='EQUIPAMENTO',
    text_col='processed'
    )

In [ ]:
samples.head()

In [ ]:
bucket = 'ons-ds-dev-entityrecognition'
samples_equipments_path = f's3://{bucket}/mpo/mvp/data/processed/equipment_samples.parquet'
samples.to_parquet(samples_equipments_path, compression='gzip')

### Marking manual actions

In [11]:
df_manualactions = preproc.get_manual_actions(
    bucket='ons-ds-dev-entityrecognition',
    path='mpo/mvp/data/raw/manual_actions'
    )

df_manualactions.head()

,termos,actions
0,Limitações impostas pela (nome da IO-OC) atend...,limitações impostas pela
1,Atender previamente as limitações para a indis...,atender previamente as limitações
2,Adotar os procedimentos para a indisponibilidade,adotar os procedimentos para a indisponibilidade
3,Limitações impostas pela indisponibilidade da(...,limitações impostas pela indisponibilidade
4,Verificar se o somatório dos fluxos nos circui...,verificar o somatório dos fluxos


In [12]:
df_manualactions = preproc.preprocess_fragments(df_manualactions, 'actions')
df_manualactions.head()

,termos,actions,processed
0,Limitações impostas pela (nome da IO-OC) atend...,limitações impostas pela,limitacoes impostas pela
1,Atender previamente as limitações para a indis...,atender previamente as limitações,atender previamente as limitacoes
2,Adotar os procedimentos para a indisponibilidade,adotar os procedimentos para a indisponibilidade,adotar os procedimentos para a indisponibilidade
3,Limitações impostas pela indisponibilidade da(...,limitações impostas pela indisponibilidade,limitacoes impostas pela indisponibilidade
4,Verificar se o somatório dos fluxos nos circui...,verificar o somatório dos fluxos,verificar o somatorio dos fluxos


In [ ]:
df_fragments_ma = preproc.find_manual_actions(
    df_fragments,
    df_manualactions
    )

df_fragments_ma.head()

---

### Create synthetic samples

In [ ]:
df_fragments_ma.astype('str').drop_duplicates(subset=['MpoCentro', 'processed', 'ACAOMANUAL'], keep='last', inplace=True)
samples_manualactions = preproc.create_synthetic_samples(
    df_fragments_ma,
    entity_col='ACAOMANUAL',
    text_col='processed'
    )
samples_manualactions.head()

In [ ]:
bucket = 'ons-ds-dev-entityrecognition'
samples_manualactions_path = f's3://{bucket}/mpo/mvp/data/processed/manualactions_samples.parquet'
samples_manualactions.to_parquet(samples_manualactions_path, compression='gzip')